In [2]:
import numpy as np
import torchvision

import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim

import math
from torch.nn import init
from torch.autograd import Variable
from tqdm import trange
from torch.distributions.categorical import Categorical
import scipy
import scipy.linalg
from collections import Counter


#from model import CnnActorCriticNetwork, RNDModel
from utils import global_grad_norm_
device = 'cuda'


SyntaxError: invalid syntax (<ipython-input-2-d338c6d9132c>, line 20)

### Dynamical Isomentry Check

In [158]:
def noise_sample(obs_batch):
    obs_batch = obs_batch.cpu().detach().numpy()
    sample = np.random.normal(size=obs_batch.shape)
    sample = torch.from_numpy(sample).float().cuda()# use .to(self.device) soon 
    return sample

def noise_sample_step(obs_batch, epsilon=1):
    obs_batch = obs_batch.cpu().detach().numpy()
    step = np.random.normal(size=obs_batch.shape)
    step = (step / np.linalg.norm(step)) * epsilon
    z_obs_batch = obs_batch + step
    z_obs_batch = torch.from_numpy(z_obs_batch).float().cuda()# use .to(self.device) soon 
    return z_obs_batch

In [191]:
def init_weights(m):
    if type(m)==nn.Linear:
        init.orthogonal_(m.weight)

In [192]:
lnn =  nn.Sequential(nn.Linear(784, 784))

In [193]:
lnn.apply(init_weights)

Sequential(
  (0): Linear(in_features=784, out_features=784, bias=True)
)

In [194]:
w = lnn[0].weight.detach().cpu().numpy()
u,s,v = scipy.linalg.svd(w)
print(np.mean(s))

1.0


###  Plan 
#### Train model on 10 samples from class 0 - Done
#### Create Model with separated predictors - Done
#### Train/Test Model with separated predictors - Done
#### Debug Model with separated predictors - In progress

In [47]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data/MNIST/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=False)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data/MNIST/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=False)

In [53]:
class AEModel(nn.Module):
    def __init__(self, n_classes):
        super(AEModel, self).__init__()
        
        self.activated_predictor = None
        
        #self.target =  nn.Sequential(nn.Linear(784, 784))
        
        self.predictors = {}
        for c in range(n_classes):
            self.predictors['predictor_'+str(c)] = nn.Sequential(
            nn.Linear(28 * 28, 28 * 28),
            #nn.ReLU(True),
            #nn.Linear(128, 64),
            #nn.ReLU(True), 
            #nn.Linear(64, 12), 
            #nn.ReLU(True), 
            #nn.Linear(12, 3),
            #nn.Linear(3, 12),
            #nn.ReLU(True),
            #nn.Linear(12, 64),
            #nn.ReLU(True),
            #nn.Linear(64, 128),
            #nn.ReLU(True), 
            #nn.Linear(128, 28 * 28))
            #nn.Tanh())
            
        
        for p in self.modules():
        #    if isinstance(p, nn.Conv2d):
        #        init.orthogonal_(p.weight, np.sqrt(2))
        #        p.bias.data.zero_()

            if isinstance(p, nn.Linear):
                #init.orthogonal_(p.weight, np.sqrt(2))
                init.orthogonal_(p.weight)
                #init.orthogonal_(p.bias)
                #p.bias.data.zero_()

        for predictor in self.predictors:
            for param in self.predictors[predictor].parameters():
                param.requires_grad = False

                
    def cuda_predictors(self):
        for predictor in self.predictors:
            self.predictors[predictor].cuda()
                
                
    def activate_predictor(self, class_):
        self.activated_predictor = self.predictors['predictor_'+str(class_)]
        for param in self.activated_predictor.parameters():
            param.requires_grad = True
                
    def deactivate_predictor(self):
        for param in self.activated_predictor.parameters():
            param.requires_grad = False
            
            
    def predict(self, next_obs):
        predict_features = []
        #target_feature = self.target(next_obs)
        for predictor in self.predictors:
            predict_features.append(self.predictors[predictor](next_obs))
        return predict_features
            
            
    def forward(self, next_obs):
        #target_feature = self.target(next_obs)
        predict_feature = self.activated_predictor(next_obs)

        return predict_feature

SyntaxError: invalid syntax (<ipython-input-53-3cc9858d024e>, line 29)

In [56]:
rnd = AEModel(10)
rnd.to(device)
rnd.cuda_predictors()
print(rnd)

params =[]
for _, predictor in rnd.predictors.items():
    params += list(predictor.parameters())

optimizer = optim.Adam(params,lr=0.001)
forward_mse = nn.MSELoss(reduction='none')

update_proportion = 0.25


#Batch size must be 1!
def train(epoch, rnd, train_loader, shots_num):
    for batch_idx, (data, y) in enumerate(train_loader):
        data = data.view(data.shape[0],-1 )
        rnd.activate_predictor(class_=y.cpu().numpy()[0])

        predict_next_state_feature  = rnd(Variable(data.to(device)))
        forward_loss = forward_mse(predict_next_state_feature, data.to(device).detach()).mean(-1)
        forward_loss = forward_loss.sum()/len(forward_loss)

        #Some unknown rnd regularization!
        #mask = torch.rand(len(forward_loss)).to(device)
        #mask = (mask < update_proportion).type(torch.FloatTensor).to(device)
        #forward_loss = (forward_loss * mask).sum() / torch.max(mask.sum(), torch.Tensor([1]).to(device))
        
        #params =[]
        #for _, predictor in rnd.predictors.items():
        #    params += list(predictor.parameters())
        #print('Now using predictor number ', y.cpu().numpy()[0])
        #print('params before update for predictor 0:', params[0])
        #print('params before update for predictor 1:', params[4])
        
        optimizer = optim.Adam(list(rnd.activated_predictor.parameters()),lr=0.001)
        optimizer.zero_grad()
        loss = forward_loss
        loss.backward()
        global_grad_norm_(list(rnd.activated_predictor.parameters()))
        optimizer.step()
        
        #params =[]
        #for _, predictor in rnd.predictors.items():
        #    params += list(predictor.parameters())
        #print('params after update for predictor 0:', params[0])
        #print('params after update for predictor 1:', params[4])

        #rnd.deactivate_predictor()

        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), shots_num,
            100. * batch_idx / shots_num, loss.item()))
          #train_losses.append(loss.item())
        
        
def test(rnd, test_loader, shots_num=1000):
    rnd.eval()
    test_loss = 0
    correct = 0
    mses = []
    with torch.no_grad():
        for batch_idx, (data, y)  in enumerate(test_loader): 
            data = data.view(data.shape[0],-1 )
            predict_next_state_feature = rnd.predict(Variable(data.to(device)))
            for predict in predict_next_state_feature:
                mses.append((data.to(device) - predict).pow(2).sum(1) / 2)
            min_mse = np.argmin(mses)
            #print('min_mse',min_mse)
            #print('y',y.cpu().numpy()[0])
            if min_mse==y.cpu().numpy()[0]:
                correct+=1
            mses = []
        print('Accuracy: {}/{} ({:.0f}%)\n'.format(correct, batch_idx+1, 100. * correct / (batch_idx+1)))
        #len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    #return(test_loss)

AEModel()


### Full MNIST

In [44]:
num_of_samples = 200
few_shot_dataset = []
few_shot_dataset_y = []
for batch_idx, (data, target) in enumerate(train_loader):
    few_shot_dataset.append(data)
    few_shot_dataset_y.append(target)
    if len(few_shot_dataset)>num_of_samples:
        break

In [49]:
num_of_shots = 11
break_treshold = num_of_shots*20
few_shot_dataset = []
few_shot_dataset_y = []
few_shot_dataset_y_np = list(range(0,10))
for batch_idx, (data, target) in enumerate(train_loader):
    num_of_samples = [x for x in Counter(few_shot_dataset_y_np).values()]
    pos_of_samples = [x for x in Counter(few_shot_dataset_y_np).keys()]
    if num_of_samples[pos_of_samples.index(target.cpu().numpy()[0])]<num_of_shots:
        few_shot_dataset.append(data)
        few_shot_dataset_y.append(target)
        few_shot_dataset_y_np.append(target.cpu().numpy()[0])
    if batch_idx>break_treshold:
        break

In [13]:
test(rnd, test_loader)

Accuracy: 817/10000 (8%)



In [57]:
for epoch in range(1, 500 + 1):
    train(epoch, rnd, zip(few_shot_dataset, few_shot_dataset_y), len(few_shot_dataset))
    #if epoch % 10==0:
    test(rnd, test_loader)

Train Epoch: 1 [0/100 (0%)]	Loss: 1.169958
Accuracy: 6528/10000 (65%)

Train Epoch: 2 [0/100 (0%)]	Loss: 0.922739
Accuracy: 6882/10000 (69%)

Train Epoch: 3 [0/100 (0%)]	Loss: 0.747401
Accuracy: 7003/10000 (70%)

Train Epoch: 4 [0/100 (0%)]	Loss: 0.594257
Accuracy: 7123/10000 (71%)

Train Epoch: 5 [0/100 (0%)]	Loss: 0.486534
Accuracy: 7226/10000 (72%)

Train Epoch: 6 [0/100 (0%)]	Loss: 0.451475
Accuracy: 7315/10000 (73%)

Train Epoch: 7 [0/100 (0%)]	Loss: 0.419226
Accuracy: 7257/10000 (73%)

Train Epoch: 8 [0/100 (0%)]	Loss: 0.386739
Accuracy: 7366/10000 (74%)

Train Epoch: 9 [0/100 (0%)]	Loss: 0.351120
Accuracy: 7334/10000 (73%)

Train Epoch: 10 [0/100 (0%)]	Loss: 0.325390


KeyboardInterrupt: 